<a href="https://colab.research.google.com/github/ashfaq-polit/Large_language_models/blob/master/LLM_Evaluation_TruLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [19]:
# !pip install llama-index-llms-langchain

In [9]:
# !pip install -q trulens_eval
!pip install "trulens-apps-llamaindex>=1.0.0"

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
# from llama_index.llms.langchain import LangChainLLM

from google.colab import userdata
hf_token = userdata.get("llama3")  # Make sure it's stored correctly

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.9,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipython-input-1-1345738778.py:30: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # or any other HF-compatible model
    device="cuda" if torch.cuda.is_available() else "cpu"
)


In [2]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf


from pypdf import PdfReader, PdfWriter
import io

def select_pdf_pages(file, pages):
    pdf_reader = PdfReader(file)
    pdf_writer = PdfWriter()
    for page in pages:
        pdf_writer.add_page(pdf_reader.pages[page])

    buf = io.BytesIO()
    pdf_writer.write(buf)
    buf.seek(0)
    return buf

new_pdf = select_pdf_pages(
    open("IPCC_AR6_WGII_Chapter03.pdf", "rb"), list(range(0, 30))
)
with open("IPCC_AR6_WGII_Chapter03_subset.pdf", "wb") as f:
    f.write(new_pdf.getvalue())


In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceWindowNodeParser

# Set global defaults for LLM and embedding model
Settings.llm = llm
Settings.embed_model = embed_model

# Load documents
documents = SimpleDirectoryReader(input_files=["IPCC_AR6_WGII_Chapter03_subset.pdf"]).load_data()

# Build basic index using updated approach
def build_basic_index(documents):
    return VectorStoreIndex.from_documents(documents)


def build_sentence_window_index(documents):
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original-text"
    )
    # Use Settings instead of ServiceContext
    Settings.node_parser = node_parser
    return VectorStoreIndex.from_documents(documents=documents)

In [6]:
from trulens_eval import TruLlama, Tru

tru = Tru()

def get_trulens_recorder(query_engine, app_id):
    return TruLlama(query_engine, app_id=app_id)


/tmp/ipython-input-6-1457315951.py:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import TruLlama, Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [7]:
import pandas as pd
qa_df = pd.read_csv("ipcc_test_questions.csv")
qa_set = [{"query": row["Question"], "response": row["Answer"]} for _, row in qa_df.iterrows()]

len(qa_set)


10

In [8]:
basic_query_index = build_basic_index(documents)
basic_query_engine = basic_query_index.as_query_engine()

recorder = get_trulens_recorder(basic_query_engine, app_id="LLaMA3_Basic")
with recorder as recording:
    for q in qa_set[:4]:
        print("Query:", q["query"])
        print("Answer:", basic_query_engine.query(q["query"]))

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

In [10]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
sentence_window_index = build_sentence_window_index(documents=documents)

postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

sentence_query_engine = sentence_window_index.as_query_engine(
    node_postprocessors=[postproc]
)

sentence_recorder = get_trulens_recorder(
    sentence_query_engine, app_id="Sentence Window Query Engine"
)
with sentence_recorder as recording:
    for q in qa_set[:4]:
        sentence_query_engine.query(q['query'])

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [24]:
# !pip uninstall -y trulens_eval
# !pip install -qU trulens_eval
!pip install trulens-providers-huggingface



In [15]:
# !pip install -qU trulens_eval sentence-transformers
# !pip install --quiet "trulens_eval>=0.24.0"


from trulens_eval import Tru, Feedback, TruLlama
from trulens.providers.huggingface import Huggingface
import numpy as np

# Initialize TruLens
tru = Tru()

# Initialize HuggingFace provider
huggingface_provider = Huggingface(model="sentence-transformers/all-MiniLM-L6-v2")

# Define feedback function
f_qa_relevance = Feedback(
    huggingface_provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()


# Context relevance (source-grounding)
f_qs_relevance = Feedback(
    huggingface_provider.relevance_with_cot_reasons, name="Context Relevance"
).on_input().on(TruLlama.select_source_nodes().node.text).aggregate(np.mean)


AttributeError: 'Huggingface' object has no attribute 'relevance_with_cot_reasons'